In [1]:
#using direct sampling by random normal function, instead of MH for conditional distribution
#allow double zero at first

In [2]:
import numpy as np
import numpy.random as random
import matplotlib.pyplot as plt

In [3]:
#from here on, direct sampling on w
def generate_std_mu_w(index,w,pi_l,pi_r,var_l,var_r,x_l,x_r,b,tau,beta,dim,p):
    ind_0_low,ind_0_high = int(index[0]-(np.sign(index[0]-0.99)+1)/2),int(index[0]+(np.sign((dim-1-0.01)-index[0])+1)/2)
    ind_1_low,ind_1_high = int(index[1]-(np.sign(index[1]-0.99)+1)/2),int(index[1]+(np.sign((dim-1-0.01)-index[1])+1)/2)
    no_nei = ind_0_high - ind_0_low + ind_1_high - ind_1_low
    sum_nei = np.sum(w[:,index[0],ind_1_low:ind_1_high+1],axis=1)-2*w[:,index[0],index[1]]+np.sum(w[:,ind_0_low:ind_0_high+1,index[1]],axis=1)
    std = (2*tau*((x_l[:,index[0],index[1]]-x_r[:,index[0],index[1]])**2+beta*no_nei+pi_l[:,index[0],index[1]]/2/var_l+pi_r[:,index[0],index[1]]/2/var_r))**-0.5
    mu = tau*((b[:,index[0],index[1]]-x_r[:,index[0],index[1]])*(x_l[:,index[0],index[1]]-x_r[:,index[0],index[1]])+(x_l[:,index[0],index[1]]-x_r[:,index[0],index[1]])**2/2/var_l*pi_l[:,index[0],index[1]]+beta*sum_nei)
    mu = mu/(0.5*std**-2)
    return mu,std

In [4]:
#direct sampling on w tested

In [5]:
#binary sampling from this point on

In [6]:
p_1_r = 0.8
l_sample = []
for i in range(1000000):
    if random.random()<p_1_r/(1+p_1_r):
        l_sample.append(1)
    else:
        l_sample.append(0)
print(sum(l_sample))
print(0.8/1.8)

445422
0.4444444444444445


In [7]:
def p_pi_l_new(index,w,x_l,x_r,pi_l,var,b,tau,gamma,p):
    #sparse coefficient alpha TBC
    i,j = index[0],index[1]
    s_n = w[:,i,j]*x_l[:,i,j]+(1-w[:,i,j])*x_r[:,i,j]
    E_xl = 1/(2*var)*pi_l[:,i,j]*(x_l[:,i,j]-s_n)**2
    pi_extended = np.concatenate((pi_l[:,:,0].reshape(p,-1,1),pi_l,pi_l[:,:,-1].reshape(p,-1,1)),axis=2)
    pi_extended = np.concatenate((pi_extended[:,0,:].reshape(p,1,-1),pi_extended,pi_extended[:,-1,:].reshape(p,1,-1)),axis=1)
    #E_pi_l = alpha*(pi_extended[i+2,j+1]+pi_extended[i+1,j+2]+pi_extended[i,j+1]+pi_extended[i+1,j])
    '''E_pi_l = E_pi_l + gamma*((pi_extended[:,i+1,j+1]-pi_extended[:,i,j+1])**2+(pi_extended[:i+1,j+1]-pi_extended[:,i+1,j])**2)'''
    E_pi_l = gamma*((pi_extended[:,i+1,j+1]-pi_extended[:,i,j+1])**2+(pi_extended[:,i+1,j+1]-pi_extended[:,i+1,j])**2)
    E_pi_l = E_pi_l + gamma*((pi_extended[:,i+1,j+1]-pi_extended[:,i+2,j+1])**2+(pi_extended[:,i+1,j+1]-pi_extended[:,i+1,j+2])**2)
    return np.exp(-tau*(E_xl+E_pi_l))

In [8]:
def sample_rejection_pi_l_new(index,w,x_l,x_r,pi_l,pi_l_star,var,b,tau,gamma,p):
    p_pi_l_cur = p_pi_l_new(index,w,x_l,x_r,pi_l,var,b,tau,gamma,p)
    p_pi_l_star = p_pi_l_new(index,w,x_l,x_r,pi_l_star,var,b,tau,gamma,p)
    p_acc = p_pi_l_star/p_pi_l_cur
    p_ran = random.random(p)
    acc = np.sign(p_acc-p_ran).astype('int')
    return acc

def binary_proposal_new(z,index):
    #p: the probability of switch, to stablize the system
    z_star = z.copy()
    z_star[:,index[0],index[1]] = 1-z[:,index[0],index[1]]
    return z_star

In [9]:
def likelihood_ratio_pi_l(index,w,x_l,x_r,pi_l,var,b,tau,gamma,p):
    #likelihood ratio: p(pi_l)=1/p(pi_l)=0
    pi_l_1 = pi_l.copy()
    pi_l_1[:,index[0],index[1]] = 1
    pi_l_0 = pi_l.copy()
    pi_l_0[:,index[0],index[1]] = 0
    return p_pi_l_new(index,w,x_l,x_r,pi_l_1,var,b,tau,gamma,p)/p_pi_l_new(index,w,x_l,x_r,pi_l_0,var,b,tau,gamma,p)

In [10]:
index = (1,1)
p = 200
no_samples = 80000
w = np.array([random.random((5,5))]*p)
w_1 = w.copy()
w_2 = w.copy()
x_l = np.ones((p,5,5))
x_r = np.zeros((p,5,5))
pi_l_o = np.ones((p,5,5))
pi_l_1 = pi_l_o.copy()
pi_l_2 = pi_l_o.copy()
pi_r = np.ones((p,5,5))
beta = 10
gamma =10
tau = 1/100
var = 0.001
b = 0.5*np.ones((p,5,5))
pi_l_list = []
for i in range(no_samples):
    pi_l_star = binary_proposal_new(pi_l_1, index)
    acc_2 = sample_rejection_pi_l_new(index,w_1,x_l,x_r,pi_l_1,pi_l_star,var,b,tau,gamma,p)
    for j in range(p):
        if acc_2[j] == 1:
            pi_l_1[j,:,:] = pi_l_star[j,:,:]
    pi_l_i = pi_l_1.copy()
    pi_l_list.append(pi_l_i)


In [12]:
ratio = likelihood_ratio_pi_l(index,w_2,x_l,x_r,pi_l_2,var,b,tau,gamma,p)
print(ratio/(1+ratio))

[0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283
 0.41273283 0.41273283 0.41273283 0.41273283 0.41273283 0.4127